## 音声認識モデル Whisper を SageMaker 上でデプロイして試してみる



## 環境のセットアップ

### モジュールのインストール

In [10]:
!pip install -U boto3
!pip install -U sagemaker
!pip install -U transformers

You should consider upgrading via the '/home/studio-lab-user/.conda/envs/default/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/studio-lab-user/.conda/envs/default/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 5.3 MB 3.9 MB/s            
     |████████████████████████████████| 163 kB 63.2 MB/s            
     |████████████████████████████████| 7.6 MB 42.1 MB/s            
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.19
    Uninstalling huggingface-hub-0.0.19:
      Successfully uninstalled huggingface-hub-0.0.19
  Attempting uninstall: transformers
    Found existing installation: transformers 4.12.2
    Uninstalling transformers-4.12.2:
      Successfully uninstalled transformer


### IAM ユーザーの作成と認証情報の設定
今回のサンプルでは、Studio Lab で動かしている Notebook 上ではなく AWS 環境上で Whisper をデプロイして使い方を確かめてみます。そのためには、Notebook から AWS 環境にアクセスする必要があるためその認証情報をこれから設定します。

IAM ユーザーを作成し、そこから得られるアクセスキーとシークレットキーを登録します。

まずは、AWS のコンソール画面を開いて左上の検索窓で「IAM」と検索します。トップに出てくる IAM をクリックして IAM のサービスページを開きます。

![](./imgs/001_IAM_search.png)

左側メニューから、「ユーザー」をクリックして IAM ユーザーの設定画面に遷移します。

![](./imgs/002_IAM_user.png)

次に、「ユーザーを追加」をクリックしてユーザーの作成を開始します。

![](./imgs/003_user_create.png)

ユーザー名に「whisper-sample-user」（他の名称でも大丈夫です）、「アクセスキー - プログラムによるアクセス」にチェックをつけます。

![](./imgs/004_user_info.png)

「次のステップ」をクリックします。  
その後「既存のポリシーを直接アタッチ」を選択し、ポリシーの検索で「SageMakerFullAccess」と入力します。そうすると「AWSSageMakerFullAccess」のポリシー候補が現れるのでこれを選択します。

![](./imgs/005_user_policy.png)

次に、検索窓に「PowerUserAccess」と検索し候補に出てきた「PowerUserAccess」を選択します。  

![](./imgs/006_user_poweruseraccess.png)

「次のステップ」をクリックするとタグの設定画面が出てきますが、ここは特に入力せずにスキップします。  

これまでに設定した項目の確認ページが出てくるので問題なければ「ユーザーの作成」をクリックします。

![](./imgs/007_user_confirmation.png)  

無事ユーザーが作成されるとユーザーキーとシークレットキーが表示されるのでメモに残しておきます。これらの情報を使って Studio Lab 経由で AWS 環境にアクセスを行います。  
**ここで取得されるクレデンシャル情報の扱いには十分注意してください**。

![](./imgs/008_user_credentials.png)


次に、 Studio Lab の画面に戻って先ほど取得したアクセスキーなどの情報を登録していきます。  

画面上部のメニューから 「File -> New -> Terminal」 と選択してターミナルの起動をします。  
![](./imgs/009_start_terminal.png)

開かれたターミナルで `aws configure` を実行します。
そこでアクセスキーとシークレットキーを聞かれるので先ほどメモした値を入力します。  

![](./imgs/010_aws_configure.png)

以上で、認証情報の設定は完了です。ではこれから実際にモデルを動かしていきましょう。

### SageMaker Inference Instance が使用する IAM ロールを作成する

今度は、SageMaker 側でモデルをデプロイする際にデプロイされたインスタンスに付与される IAM ロールを作成します。  

下記のハンズオン資料の「2-3. SageMaker Training Instance が利用する IAM ロールを作成する」で紹介されている手順とほぼ同じです。  
- https://github.com/aws-samples/aws-ml-enablement-workshop/blob/main/notebooks/scenario_churn/customer_churn_sagemaker.ipynb 

AWS のコンソール画面に戻ります。  
先ほどと同様の手順で IAM のサービス画面を開き、「ロール」を左側のメニューから選択します。 
IAM ロールの画面が開かれたら「ロールの作成」ボタンをクリックします。  

![](./imgs/011_role_create.png)  

ロールの作成画面が表示されたら信頼されるエンティティタプとして「AWS のサービス」を選択し、ユースケースのところは下の検索欄から「SageMaker」などと検索して SageMaker を選択します。

![](./imgs/012_role_entity.png)

「次へ」をクリックし、「AmazonSageMakerFullAccess」のポリシーがアタッチされていることを確認します。  

![](./imgs/013_role_policy.png)

「次へ」をクリックし、Role 名を設定します。「StudioLabWhisperExecutionRole」と入力し、他の項目はいじらずに「ロールを作成」をクリックします。

![](./imgs/014_role_name.png)

作成した IAM ロールのリソースネームである ARN を取得します。  
IAM ロールの画面から、検索欄で「StudioLabWhisper」などと入力して先ほど作成した IAM ロールを探して選択します。  

![](./imgs/015_role_search.png)  

IAM ロールの詳細情報が表示されるので、ARN の隣にあるコピーボタンをクリックして ARN をコピーします。
![](./imgs/016_role_arn_copy.png)  

コピペした値を置き換えて role の値を設定します。

In [14]:
role = "arn:aws:iam::392304288222:role/StudioLabWhisperExecutionRole"  # コピペした値で置き換える

## Whisper のデプロイ

まずは、 SageMaker SDK の HuggingFace 拡張を使って簡単にモデルをデプロイしてみましょう。  
以下のページで公開されている「whisper-base」と呼ばれているモデルを使っていきます。  

- https://huggingface.co/openai/whisper-base

この HuggingFace のページ上で SageMaker でモデルをデプロイするためのコードを手軽に生成できます。  

まずは、ページにある「Deploy」ボタンをクリックします。  

![](./imgs/101_deploy_button.png)

いくつかデプロイの選択肢が出てくるので今回は「Amazon SageMaker」を選択します。  

![](./imgs/102_deploy_select_sagemaker.png)

Task を「Automatic Speech Recognition」、Configuration を「AWS」に設定すると deploy 用のコードが生成されます。  

![](./imgs/103_deploy_generate_code.png)



コピーしたコードはデプロイのコードと推論のコードが含まれています。  
推論部分はいじる必要があるため、コメントアウトします。また、デプロイ部分も IAM ロールの取得部分だけコメントアウトをしておきます。

```python
from sagemaker.huggingface import HuggingFaceModel
import sagemaker

# 下の行をコメントアウト。先ほど作成したロールを使う。
# role = sagemaker.get_execution_role()
# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'openai/whisper-base',
	'HF_TASK':'automatic-speech-recognition'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	transformers_version='4.23.1',  # バージョンは 4.23 以降に
	pytorch_version='1.10.2',
	py_version='py38',
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.m5.xlarge' # ec2 instance type
)

# 推論部分も別途実装するためコメントアウト
# predictor.predict({
# 	'inputs': "sample1.flac"
# })
```


In [44]:
!mkdir code

In [120]:
%%writefile code/inference.py

from transformers import pipeline
from transformers.pipelines import AutomaticSpeechRecognitionPipeline
import numpy as np

def model_fn(model_dir) -> AutomaticSpeechRecognitionPipeline:
    return pipeline(model="facebook/wav2vec2-base-960h")  

def predict_fn(data, pipeline):
    inputs = data.pop("inputs", data)
    parameters = data.pop("parameters", None)
    if type(inputs) == list:
        inputs = np.array(inputs, dtype=np.float)
    print("inputs are: ", inputs)
    # pass inputs with all kwargs in data
    if parameters is not None:
        prediction = pipeline(inputs, **parameters)
    else:
        prediction = pipeline(inputs)
    return prediction

Overwriting code/inference.py


In [121]:
# https://github.com/huggingface/notebooks/blob/main/sagemaker/17_custom_inference_script/sagemaker-notebook.ipynb を参考に
s3_bucket_name = "sagemaker-ap-northeast-1-392304288222"
# repository = "openai/whisper-tiny"
repository = "facebook/wav2vec2-base-960h"
model_id = repository.split("/")[-1]
s3_location = f"s3://{s3_bucket_name}/custom_inference/{model_id}/model.tar.gz"

In [122]:
%cd ~/aws-ml-jp/sagemaker/studio-lab-whisper

/home/studio-lab-user/aws-ml-jp/sagemaker/studio-lab-whisper


In [123]:
!git lfs install
!git clone https://huggingface.co/$repository

Updated git hooks.
Git LFS initialized.
fatal: destination path 'wav2vec2-base-960h' already exists and is not an empty directory.


In [124]:
!cp -r code/ $model_id/code/

In [125]:
%cd $model_id
!rm model.tar.gz
!tar zcvf model.tar.gz *

/home/studio-lab-user/aws-ml-jp/sagemaker/studio-lab-whisper/wav2vec2-base-960h
README.md
code/
code/inference.py
code/.ipynb_checkpoints/
code/code/
code/code/inference.py
config.json
feature_extractor_config.json
preprocessor_config.json
pytorch_model.bin
special_tokens_map.json
tf_model.h5
tokenizer_config.json
vocab.json


圧縮したモデルと推論コードを S3 にアップロード

In [126]:
!aws s3 cp model.tar.gz $s3_location

upload: ./model.tar.gz to s3://sagemaker-ap-northeast-1-392304288222/custom_inference/wav2vec2-base-960h/model.tar.gz


In [127]:
huggingface_model = HuggingFaceModel(
    model_data=s3_location,
    role=role,
    transformers_version="4.17.0",
    pytorch_version="1.10.2",
    py_version="py38"
)

predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.2xlarge"
)

-----!

`ffmpeg -i <infile> -ac 2 -f wav <outfile>`

## デプロイしたモデルに音声認識させてみる

では次に、デプロイしたモデルに対して音声認識をさせてみましょう。

In [130]:
import scipy

In [140]:
wav_file = scipy.io.wavfile.read("audio_message.wav")

In [142]:
wav_file[1].shape

(219136,)

In [143]:
input_array = np.random.randn(10000)

predictor.predict({
    'inputs': wav_file[1]
})

{'text': 'UROUND ME ON YOU E'}

In [15]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker

# 下の行をコメントアウト。先ほど作成したロールを使う。
# role = sagemaker.get_execution_role()
# Hub Model configuration. https://huggingface.co/models
hub = {
    # 'HF_MODEL_ID':'openai/whisper-base',
    'HF_MODEL_ID': 'facebook/wav2vec2-base-960h',
    'HF_TASK':'automatic-speech-recognition'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    transformers_version='4.17.0',
    pytorch_version='1.10.2',
    py_version='py38',
    env=hub,
    role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1, # number of instances
    instance_type='ml.m5.xlarge' # ec2 instance type
)

# predictor.predict({
#     'inputs': "sample1.flac"
# })

-----!

In [43]:
predictor.serializer = sagemaker.serializers.NumpySerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

In [48]:
type(pipe)

transformers.pipelines.automatic_speech_recognition.AutomaticSpeechRecognitionPipeline

In [42]:
import json
input_array = np.random.randn(1, 10000)

predictor.predict({
    'inputs': input_array
})

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from primary with message "{
  "code": 400,
  "type": "InternalServerException",
  "message": "\u0027numpy.ndarray\u0027 object has no attribute \u0027pop\u0027"
}
". See https://ap-northeast-1.console.aws.amazon.com/cloudwatch/home?region=ap-northeast-1#logEventViewer:group=/aws/sagemaker/Endpoints/huggingface-pytorch-inference-2022-10-19-05-44-57-194 in account 392304288222 for more information.

In [3]:
import numpy as np

In [8]:
import torch
torch.__version__

'1.10.0'

In [2]:
!pip install -U datasets
!pip install -U soundfile

You should consider upgrading via the '/home/studio-lab-user/.conda/envs/default/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/studio-lab-user/.conda/envs/default/bin/python -m pip install --upgrade pip' command.


In [8]:
import datasets
datasets.__version__

'1.6.2'

In [4]:
import numpy as np
input_array = torch.from_numpy(np.random.randn(10000))

In [5]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset
import torch

# load model and processor

processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

# load dummy dataset and read soundfiles

# ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
input_features = processor(input_array, return_tensors="pt").input_features 

# Generate logits

logits = model(input_features, decoder_input_ids = torch.tensor([[50258]])).logits 

# take argmax and decode

predicted_ids = torch.argmax(logits, dim=-1)

transcription = processor.batch_decode(predicted_ids)

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [8]:
from transformers import pipeline

pipe = pipeline(model="openai/whisper-tiny")

Downloading:   0%|          | 0.00/3.85k [00:00<?, ?B/s]

In [12]:
pipe(np.random.randn(10000))

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/transformers/generation_utils.py:1296: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'text': ' you'}

In [9]:
pipe?

Signature:      pipe(inputs: Union[numpy.ndarray, bytes, str], **kwargs)
Type:           AutomaticSpeechRecognitionPipeline
String form:    <transformers.pipelines.automatic_speech_recognition.AutomaticSpeechRecognitionPipeline object at 0x7f915021d400>
File:           ~/.conda/envs/default/lib/python3.9/site-packages/transformers/pipelines/automatic_speech_recognition.py
Docstring:     
Pipeline that aims at extracting spoken text contained within some audio.

The input can be either a raw waveform or a audio file. In case of the audio file, ffmpeg should be installed for
to support multiple audio formats

Arguments:
    model ([`PreTrainedModel`] or [`TFPreTrainedModel`]):
        The model that will be used by the pipeline to make predictions. This needs to be a model inheriting from
        [`PreTrainedModel`] for PyTorch and [`TFPreTrainedModel`] for TensorFlow.
    tokenizer ([`PreTrainedTokenizer`]):
        The tokenizer that will be used by the pipeline to encode data for the 

In [7]:
logits.shape

torch.Size([1, 1, 51865])

In [6]:
transcription

['<|nocaptions|>']

In [9]:
import transformers
transformers.__version__

'4.12.2'

In [5]:
predictor

In [11]:
predictor.delete_endpoint()

In [14]:
# !pip install torchaudio ipywebrtc
# !conda install -c conda-forge ffmpeg
!jupyter nbextension enable --py widgetsnbextension

Exception while loading config file /etc/jupyter/jupyter_notebook_config.py
    Traceback (most recent call last):
      File "/etc/jupyter/jupyter_notebook_config.py", line 2, in <module>
        from amzn_sagemaker_studiolab.managers.kernelspec_managers import SageMakerStudioLabKernelSpecManager as KernelSpecManager
    ModuleNotFoundError: No module named 'amzn_sagemaker_studiolab'
    
    During handling of the above exception, another exception occurred:
    
    Traceback (most recent call last):
      File "/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/traitlets/config/application.py", line 738, in _load_config_files
        config = loader.load_config()
      File "/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/traitlets/config/loader.py", line 614, in load_config
        self._read_file_as_dict()
      File "/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/traitlets/config/loader.py", line 646, in _read_file_

In [4]:
!conda install -c conda-forge python-sounddevice

failed

CondaError: KeyboardInterrupt



In [1]:
from ipywebrtc import AudioRecorder, CameraStream
import torchaudio
from IPython.display import Audio

camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [3]:
!ffmpeg

/usr/bin/sh: 1: ffmpeg: not found


In [2]:
with open('recording.webm', 'wb') as f:
    f.write(recorder.audio.value)
!ffmpeg -i recording.webm -ac 1 -f wav file.wav -y -hide_banner -loglevel panic
sig, sr = torchaudio.load("file.wav")
print(sig.shape)
Audio(data=sig, rate=sr)

/usr/bin/sh: 1: ffmpeg: not found


RuntimeError: Failed to load audio from file.wav

In [10]:
from IPython.display import Javascript
from base64 import b64decode

RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec, filename='audio.wav'):
  display(Javascript(RECORD))
  
  s = output.eval_js('record(%d)' % (sec * 1000))
  b = b64decode(s.split(',')[1])
  with open(filename, 'wb+') as f:
    f.write(b)


In [11]:
record(30)

<IPython.core.display.Javascript object>

NameError: name 'output' is not defined